<a href="https://colab.research.google.com/github/Igorps023/Amazon-Products-Review/blob/main/Music_Stream.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ETL e Análise Exploratória Streaming de Música

# Código

In [24]:
# Desmontar o Google Drive (se já estiver montado)
from google.colab import drive
drive.flush_and_unmount()

# Montar o Google Drive novamente
drive.mount('/content/drive')

Mounted at /content/drive


In [25]:
# instalar as dependências
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.3.1//spark-3.3.1-bin-hadoop2.tgz
!tar xf spark-3.3.1-bin-hadoop2.tgz
!pip install -q findspark

In [26]:
# configurar as variáveis de ambiente
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop2.7"

In [27]:
#importando biblioteca para reconhecer o pyspark
import findspark
findspark.init('spark-3.3.1-bin-hadoop2')

In [28]:
#escrevendo um arquivo no nosso Drive para testar o mapeamento
with open('/content/drive/My Drive/Colab Notebooks/testando_mount.txt', 'w') as f:
    f.write('content')

In [29]:
#Criando nossa primeira Sessão com o SparkSession
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Aplicação Pyspark Supply_Chain") \
    .config('spark.ui.port', '4050') \
    .getOrCreate()	

In [30]:
spark

In [31]:
#instalar extensão kaggle
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [32]:
import json 
import os
from pathlib import Path

# your api key
api_key = {
'username':"igorpereirasilva" ,
'key':"932cad93df082ec0266d416f7e420cdd"}

# uses pathlib Path
kaggle_path = Path('/root/.kaggle')
os.makedirs(kaggle_path, exist_ok=True)

# opens file and dumps python dict to json object 
with open (kaggle_path/'kaggle.json', 'w') as handl:
    json.dump(api_key,handl)

os.chmod(kaggle_path/'kaggle.json', 600)  

In [33]:
#importar biblioteca kaggle
import kaggle

In [34]:
#pathway até o dataset
kaggle.api.authenticate()
kaggle.api.dataset_download_files('undefinenull/million-song-dataset-spotify-lastfm', path='./data', unzip=True)

#URL Dataset
#https://www.kaggle.com/datasets/undefinenull/million-song-dataset-spotify-lastfm

In [35]:
# ler o arquivo JSON baixado do Kaggle
music_info_raw = spark.read.csv("./data/Music Info.csv", sep=",", header=True)
listening_history_raw = spark.read.csv("./data/User Listening History.csv", sep=",", header=True)

In [36]:
music_info_raw

DataFrame[track_id: string, name: string, artist: string, spotify_preview_url: string, spotify_id: string, tags: string, genre: string, year: string, duration_ms: string, danceability: string, energy: string, key: string, loudness: string, mode: string, speechiness: string, acousticness: string, instrumentalness: string, liveness: string, valence: string, tempo: string, time_signature: string]

In [37]:
listening_history_raw

DataFrame[track_id: string, user_id: string, playcount: string]

In [38]:
#numero colunas e linhas
num_rows = music_info_raw.count()
num_cols = len(music_info_raw.columns)

In [39]:
print('Shape: ({}, {})'.format(num_rows, num_cols))

Shape: (50683, 21)


In [40]:
#colunas
music_info_raw.columns

['track_id',
 'name',
 'artist',
 'spotify_preview_url',
 'spotify_id',
 'tags',
 'genre',
 'year',
 'duration_ms',
 'danceability',
 'energy',
 'key',
 'loudness',
 'mode',
 'speechiness',
 'acousticness',
 'instrumentalness',
 'liveness',
 'valence',
 'tempo',
 'time_signature']

In [41]:
music_info_raw.show(20, truncate=False)

+------------------+-----------------+---------------------+-----------------------------------------------------------------------------------------------------------+----------------------+------------------------------------------------------------------------------------------------------+-----+----+-----------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+
|track_id          |name             |artist               |spotify_preview_url                                                                                        |spotify_id            |tags                                                                                                  |genre|year|duration_ms|danceability|energy|key|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence|tempo  |time_signature|
+------------------+-----------------+---------------------+----------------------------------------------------------------

In [42]:
listening_history_raw.sort("user_id").show(20, truncate=False)

+------------------+----------------------------------------+---------+
|track_id          |user_id                                 |playcount|
+------------------+----------------------------------------+---------+
|TRQEBOU128F425D087|00000b722001882066dff9d2da8a775658053ea0|1        |
|TRBCDMC128F1452976|00001638d6189236866af9bbf309ae6c2347ffdc|1        |
|TRHVDTF128F428D64C|0000175652312d12576d9e6b84f600caa24c4715|1        |
|TREZYWT128F93191B7|00001cf0dce3fb22b0df0f3a1d9cd21e38385372|2        |
|TRTPXAP12903CA1A14|00001cf0dce3fb22b0df0f3a1d9cd21e38385372|2        |
|TRZPDFG128F92FDEF5|00001cf0dce3fb22b0df0f3a1d9cd21e38385372|2        |
|TRVRIBZ128F426BF71|00001cf0dce3fb22b0df0f3a1d9cd21e38385372|1        |
|TRGEIDA128F933B4B8|00001cf0dce3fb22b0df0f3a1d9cd21e38385372|1        |
|TRPUGUW128F426BF6F|00001cf0dce3fb22b0df0f3a1d9cd21e38385372|1        |
|TRKABMK128F426BF72|00001cf0dce3fb22b0df0f3a1d9cd21e38385372|2        |
|TRIYKXW128F92FDF08|00001cf0dce3fb22b0df0f3a1d9cd21e38385372|2  

In [43]:
listening_history_raw.columns

['track_id', 'user_id', 'playcount']